In [32]:
from urllib.request import urlopen
import json
import pandas as pd

In [33]:
originalurl = "https://api.weather.com/v1/location/VOBL:9:IN/observations/historical.json?apiKey=6532d6454b8aa370768e63d6ba5a832e&units=e&startDate=20100101&endDate=20100131"
baseurl = "https://api.weather.com/v1/location/VOBL:9:IN/observations/historical.json?apiKey=6532d6454b8aa370768e63d6ba5a832e&units=e&startDate="

In [34]:
import datetime

start = datetime.datetime.strptime("01-01-2010", "%d-%m-%Y")
end = datetime.datetime.strptime("01-01-2020", "%d-%m-%Y")
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]

In [35]:
urlList = []
for date in date_generated:
    start = date.strftime("%Y%m%d")
    url = baseurl+start
    urlList.append(url)

In [36]:
time = (pd.DataFrame(columns=['NULL'], index=pd.date_range('2020-01-01T06:00:00Z', '2020-01-01T17:30:00Z', freq='30T')).between_time('06:00','17:30').index.strftime('%H:%M')).to_list()

In [46]:
urls = urlList[0:3]

https://api.weather.com/v1/location/VOBL:9:IN/observations/historical.json?apiKey=6532d6454b8aa370768e63d6ba5a832e&units=e&startDate=20100102


In [102]:
import grequests as gr

C:\Users\Pradeep\Anaconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [103]:
reqs = (gr.get(url) for url in urlList)
resp = gr.imap(reqs, gr.Pool(10))

In [104]:
dataCsv = {}

In [105]:
temr = []
dewPt = []
wdir_cardinal = []
rh = []
pressure = []
wspd = []
times = []
for respose in resp:
    data = json.loads(respose.text)
    day = data['observations'][-25:-1]
    for j in range(len(day)):
        times.append(time[j])
        temr.append(day[j]['temp'])
        dewPt.append(day[j]['dewPt'])
        wdir_cardinal.append(day[j]['wdir_cardinal'])
        rh.append(day[j]['rh'])
        pressure.append(day[j]['pressure'])
        wspd.append(day[j]['wspd'])

KeyError: 'observations'

In [106]:
dataCsv['Time'] = pd.Series(times)
dataCsv['Temperature'] = pd.Series(temr)
dataCsv['DewPoint'] = pd.Series(dewPt)
dataCsv['WindDirection'] = pd.Series(wdir_cardinal)
dataCsv['Humidity'] = pd.Series(rh)
dataCsv['Pressure'] = pd.Series(pressure)
dataCsv['WindSpeed'] = pd.Series(wspd)

In [107]:
p = pd.DataFrame(dataCsv)

In [110]:
print(len(temr))

912


In [109]:
p.to_csv('nw.csv', index = False)